In [ ]:
import cv2
import mediapipe as mp

In [ ]:
import pickle
model_dict = pickle.load(open('./model.p','rb'))
model = model_dict['model']

In [ ]:
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [ ]:
labels_dict = {
    0: 'A',  1: 'B',  2: 'C',  3: 'D',  4: 'E',  5: 'F',  6: 'H',  7: 'G',  8: 'I',  9: 'J', 
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 
    20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: '1', 27: '2', 28: '3', 29: '4', 
    30: '5', 31: '6', 32: '7', 33: '8', 34: '9'
}

In [ ]:
predictions_list = []

while True:
    data_aux = []
    x_ = []
    y_ = []
    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    H, W, _ = frame.shape
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
    results = hands.process(frame_rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )
    
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)
                x_.append(x)
                y_.append(y)
                
        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10
         
        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        prediction = model.predict([np.asarray(data_aux)])
        
        predicted_character = labels_dict[int(prediction[0])]
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
    
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  # Spacebar to save prediction
        predictions_list.append(predicted_character)
        print(f"Stored prediction: {predicted_character}")
    elif key == ord('q'):  # 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()

print("Predictions List:", predictions_list)

In [ ]:
import requests

def get_meaningful_sentence(input_text):
    api_url = "https://api.groq.com/openai/v1/chat/completions"  # Replace with the actual Groq LLM API URL
    api_key = "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"  # Replace with your actual API key

    # Prepare the input text
    #input_text = "".join(predictions_list)
    prompt = (
    "I have a series of strings of uppercase letters without spaces that represent sentences. Your task is to:\n"
    "1. Break each string into meaningful words.\n"
    "2. Capitalize the first word of each sentence.\n"
    "3. Add necessary punctuation marks to make the sentences grammatically correct.\n"
    "4. Use common sense to correct any mistakes and determine the most likely sentence structure.\n\n"
    "For example:\n"
    "Input: 'HELLOHOWAREYOU'\n"
    "Output: 'Hello! How are you??'\n\n"
    "Input: 'WHATAREYOUDOING'\n"
    "Output: 'What are you doing?'\n\n"
    "Input: 'IAMFINE'\n"
    "Output: 'I am fine.'\n\n"
    "Input: 'HELLOHOWASEYOU'\n"
    "Output: 'Hello! How are you?'\n\n"
    "Input: 'IMISYOUSOMUCH'\n"
    "Output: 'I miss you so much.'\n\n"
    "Input: 'HAVAGOODAY'\n"
    "Output: 'Have a good day!'\n\n"
    "Now, please transform the following string:\n\n"
    f"'{input_text}'"
    )

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "messages": [
            {
                "role": "system",
                "content": prompt
            }
        ],
        "model": "gemma-7b-it",  
        "max_tokens": 256,
        "temperature": 0.7
    }

    response = requests.post(api_url, headers=headers, json=data)
    
    if response.status_code == 200:
        response_json = response.json()
        choices = response_json.get("choices")
        if choices:
            # Extract only the first sentence from the content
            content = choices[0].get("message").get("content")
            meaningful_sentence = content.split('.')[0].strip() + "."
            return meaningful_sentence
        else:
            print("No choices returned in the response")
            return None
    else:
        print("Error:", response.status_code, response.text)
        return None


input_text = ''.join(predictions_list)
meaningful_sentence = get_meaningful_sentence(input_text)

if meaningful_sentence:
    print(meaningful_sentence)